In [262]:
import pandas as pd
import re

from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from konlpy.tag import Hannanum, Kkma, Komoran, Okt
import time

data = pd.read_excel("../../petition/add_data_0414.xlsx")

In [263]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  792 non-null    int64  
 1   code        792 non-null    int64  
 2   sdays       789 non-null    object 
 3   edays       789 non-null    object 
 4   title       792 non-null    object 
 5   count       789 non-null    float64
 6   content     792 non-null    object 
 7   category    792 non-null    object 
 8   progress    792 non-null    object 
 9   link        792 non-null    object 
 10  person      792 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 68.2+ KB


In [264]:
data['progress'] = [re.sub("\n",'',progress) for progress in data['progress']]

In [265]:
data = data.reset_index(drop=True)
data.shape

(792, 11)

In [266]:
data[data['category']=='기타']

,Unnamed: 0,code,sdays,edays,title,count,content,category,progress,link,person
9,9,596809,2021-03-05,2021-04-04,뼈가 부러지고 인대가 찢어져도 방치하고 수술 후 걷지도 못하는 동생 알아서 복귀하라...,1181.0,['\n\t\t\t\t\t\t\t\t\t\t안녕하세요. \r\n국민에게 봉사하며 국...,기타,청원종료,https://www1.president.go.kr/petitions/596809,"['청원인kakao - ***', '']"
13,13,596813,2021-03-05,2021-04-04,전국 문화재 보존 영향검토 대상구역에 사는 서민들의 고통을 봐주십시오!,251.0,['\n\t\t\t\t\t\t\t\t\t\t대통령님과 각부처 장관님들의 국정 업무수...,기타,청원종료,https://www1.president.go.kr/petitions/596813,"['청원인naver - ***', '']"
19,19,596819,2021-03-05,2021-04-04,**(******) 사기꾼에 대해서 강력하게 처벌해주시기 바랍니다.,528.0,['\n\t\t\t\t\t\t\t\t\t\t안녕하십니까 \n\r\n저는 사기피해에 ...,기타,청원종료,https://www1.president.go.kr/petitions/596819,"['청원인naver - ***', '']"
46,46,596846,2021-03-08,2021-04-07,"LH직원 땅 매입과, 금융거래 직원 주식매입",1855.0,"['\n\t\t\t\t\t\t\t\t\t\tLH 직원들의 땅 매입과 관련해, \r\...",기타,청원종료,https://www1.president.go.kr/petitions/596846,"['청원인naver - ***', '']"
63,63,596863,2021-03-08,2021-04-07,4차재난 지원금 사각지대에서 죽억나는 저희 부모님을 살려주세요.,633.0,['\n\t\t\t\t\t\t\t\t\t\t안녕하세요. 부모님과 함께 가게를 도와드...,기타,청원종료,https://www1.president.go.kr/petitions/596863,"['청원인kakao - ***', '']"
...,...,...,...,...,...,...,...,...,...,...,...
757,757,597557,2021-04-13,2021-05-13,너무나 억울한 국민연금 분할제도를 개선해주세요.,176.0,['\n\t\t\t\t\t\t\t\t\t\t불공정한 국민연금 분할제도를 개선해주세요...,기타,청원진행중,https://www1.president.go.kr/petitions/597557,"['청원인naver - ***', '']"
758,758,597558,2021-04-13,2021-05-13,자동차 결함건,134.0,['\n\t\t\t\t\t\t\t\t\t\t안녕하세요 저는 ***입니다. \r\n작...,기타,청원진행중,https://www1.president.go.kr/petitions/597558,"['청원인kakao - ***', '']"
759,759,597559,2021-04-13,2021-05-13,고가의 **안마를 고발합니다,178.0,['\n\t\t\t\t\t\t\t\t\t\t저는농사를 짓는 50대 귀차니즘입니다\r...,기타,청원진행중,https://www1.president.go.kr/petitions/597559,"['청원인kakao - ***', '']"
770,770,597570,2021-04-13,2021-05-13,버팀목자금 3차 미지급사유,409.0,['\n\t\t\t\t\t\t\t\t\t\t버팀목자금 3차 지원금.\n\r\n문자를...,기타,청원진행중,https://www1.president.go.kr/petitions/597570,"['청원인kakao - ***', '']"


In [267]:
data = data[data['progress']=='청원진행중 ']

In [268]:
data

,Unnamed: 0,code,sdays,edays,title,count,content,category,progress,link,person
210,210,597010,2021-03-16,2021-04-15,새치기화이자맞은 서울**병원장과 수간호사급에게 az백신 재접종 요청합니다,2320.0,['\n\t\t\t\t\t\t\t\t\t\t서울 4대병원장들과 일선 의료진들 모두 ...,보건복지,청원진행중,https://www1.president.go.kr/petitions/597010,"['청원인twitter - ***', '']"
211,211,597011,2021-03-16,2021-04-15,"정현복 광양시장 재산증식 의혹, 전수 조사가 시급합니다.",1839.0,['\n\t\t\t\t\t\t\t\t\t\t지금 광양시민들은 광양시장이 오랜 세월 ...,교통/건축/국토,청원진행중,https://www1.president.go.kr/petitions/597011,"['청원인naver - ***', '']"
212,212,597012,2021-03-16,2021-04-15,제발 우리 아이들의 안전을 지켜주세요,498.0,['\n\t\t\t\t\t\t\t\t\t\t제발 우리 아이들의 안전을 지켜주세요\n...,육아/교육,청원진행중,https://www1.president.go.kr/petitions/597012,"['청원인facebook - ***', '']"
213,213,597013,2021-03-16,2021-04-15,스미싱/보이스피싱,248.0,['\n\t\t\t\t\t\t\t\t\t\t스미싱 피해를 당한 당사자입니다.\r\n...,기타,청원진행중,https://www1.president.go.kr/petitions/597013,"['청원인naver - ***', '']"
214,214,597014,2021-03-16,2021-04-15,필리핀 베이비시터(가사도우미) 근무할 수 있게 해주세요.,135.0,['\n\t\t\t\t\t\t\t\t\t\t현재 한국 출산율은 0.98명으로 OEC...,저출산/고령화대책,청원진행중,https://www1.president.go.kr/petitions/597014,"['청원인naver - ***', '']"
...,...,...,...,...,...,...,...,...,...,...,...
787,787,597587,2021-04-14,2021-05-14,대기업(**리테일)의 갑질과 화물노동자의 인권,248.0,['\n\t\t\t\t\t\t\t\t\t\t대기업의 갑질 과 화물노동자의 인권\n\...,일자리,청원진행중,https://www1.president.go.kr/petitions/597587,"['청원인kakao - ***', '']"
788,788,597588,2021-04-14,2021-05-14,"문재인 정부는 “국민의 건강 K방역” 끝까지 지켜주시고, “부정부패” 처벌하는 ""개...",490.0,['\n\t\t\t\t\t\t\t\t\t\t(국민청원)문재인 정부는 “국민의 건강 ...,정치개혁,청원진행중,https://www1.president.go.kr/petitions/597588,"['청원인kakao - ***', '']"
789,789,597589,2021-04-14,2021-05-14,다단계업체 전 대표 사기 및 배임 횡령과 서울지방국세청에 현금을 전달한 사건에 수사...,294.0,['\n\t\t\t\t\t\t\t\t\t\t다단계업체인 ***의 전 대표인 * 모 ...,기타,청원진행중,https://www1.president.go.kr/petitions/597589,"['청원인naver - ***', '']"
790,790,597590,2021-04-14,2021-05-14,코로나 거리두기 결혼식 예외조항 만들어주십시오,541.0,['\n\t\t\t\t\t\t\t\t\t\t코로나로 인해 피해를 보고 있는 많은 예...,보건복지,청원진행중,https://www1.president.go.kr/petitions/597590,"['청원인naver - ***', '']"


In [269]:
data['sdays'] = pd.to_datetime(data['sdays'])
data['edays'] = pd.to_datetime(data['edays'])

data['count'] = pd.to_numeric(data['count'])

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

data['content'] = [re.sub('|'.join(['\n','\t','\r','에','것','라고']),'',''.join(content)).strip() for content in data['content']]

data['content'] = [emoji_pattern.sub(r'', content) for content in data['content']]

In [270]:
doc_nouns_list = [' '.join(komoran.nouns(doc)) for doc in data['content']]

processing_nouns = []

for nouns_list in doc_nouns_list:
    no_lst = nouns_list.split(" ")
    print(no_lst)
    for i, v in enumerate(no_lst):
        if(len(v))<2:
            print(v)
            no_lst.pop(i)
    processing_nouns.append(' '.join(no_lst))

['서울', '대', '병원장', '일선', '의료진', '백신', '접종', '시점', '서울', '병원장', '수', '간호사', '급', '화이자', '접종', '대상', '화이자', '코로나', '직접', '촉', '격', '립', '병동', '응급실', '경우', '해당', '화이자', '접종', '초기', '화이자', '물량', '신청', '시', '현', '장', '근무자', '근무', '예정자', '배수로', '신청', '현재', '수', '간호사', '급', '아래', '간호사', '의사', '명단', '토지', '공사', '급', '범죄', '생각', '나라', '정책', '위반', '도덕적', '해이', '병원', '장과', '수', '간호사', '급', '의료진', '본인', '화이자', '접', '송', '대상', '아니', '무능', '이고', '줄', '국민', '배신', '기만', '범죄', '이', '지금', '당장', '교차', '접종', '할', '요청', '바', '화이자', '교차', '접종', '연구', '부족', '수', '이', '게시물', '일부', '내용', '국민', '청원', '요건', '위배', '관리자', '수정']
대
수
급
촉
립
시
장
수
급
급
수
급
접
줄
이
할
바
수
['지금', '광양시', '민', '광양시', '장이', '세월', '동안', '공직', '자리', '정보', '이용', '재산', '증식', '의혹', '제기', '임직원', '주택', '사건', '하등', '사실', '실망감', '아파트', '자리', '이해', '수', '곳', '주민', '센터', '건립', '값', '땅', '매입', '등', '시민', '의견', '무시', '채', '일', '진행', '정현', '복', '광양시', '장과', '시장', '가족', '자제', '소유', '땅', '주변', '도시', '계획', '도로', '개설', '남은', '시장', '소유', '잔여', '부지', '일반', '상업', 

도
겁
을
명
팀
등
자
처
자
바
회
년
앞
조
일
중
손
처
자
설
등
범
일
중
등
때
건
등
회
직
게
자
수
설
자
자
은
처
곳
정
속
폐
수
자
설
자
['혁신도시', '조성', '목적', '지방', '이전', '계획', '변경', '안', '철회', '국토', '안전', '관리', '원', '교육', '센터', '이전', '계획', '철회', '국토', '안전', '관리원', '지난해', '12월', '10일', '경남', '혁신도시', '진주', '이전', '한국시설안전공단', '김천시', '한국건설관리공사', '통합', '경남', '진주', '혁신도시', '출범', '지난해', '말', '국토', '안전', '관리원', '전국', '개', '지사', '설치', '교육', '센터', '인재', '교육원', '김천시', '이전', '지방', '이전', '계획', '변경', '안', '국토', '교통부', '제출', '사실', '국가', '균형발전', '혁신도시', '성장', '역행', '진주', '소재', '한국시설안전공단', '시설물', '안전', '유지', '관리', '사항', '국가', '시설물', '안전', '진단', '실시', '시설물', '관리', '이력', '정보화', '자료', '제공', '등', '업무', '수행', '시설물', '안전사고', '예방', '국민', '안전', '확보', '기능', '국토부', '산하', '공공', '기관', '김천', '소재', '한국건설관리공사', '전면', '책임감', '리제', '도입', '건설', '부조리', '예방', '부실시공', '방지', '책임감', '리', '선도', '역할', '수행', '목표', '설립', '시기', '민영화', '대상', '분류', '차례', '매각', '시도', '유', '차', '실패', '한국건설관리공사', '한국', '건설', '안전', '공단', '대체', '설립', '방안', '추진', '중단', '국토부', '산하', '공공', '기관', '국토',

원
안
수
['안녕하십니까', '김포', '한강신도시', '원주민', '비통', '일', '도움', '필요', '글', '감사', '때', '년', '김포', '한강신도시', '택지', '개발', '사업', '인해', '제가', '땅', '수용', '공익사업', '토지', '취득', '보상', '법률', '의거', '주자', '택지', '공급', '분양', '생활', '대책', '용지', '손실보상', '공급', '계약', '계약금', '중도금', '년', '개월', '회', '분납', '내야', '담보', '대출', '안', '때문', '계약자', '신용', '대출', '계약', '이자', '담보', '대출', '배', '계약', '인내', '대출', '3년', '개월', '동안', '이율', '이자', '년', '개월', '때', '토지', '사용', '불가', '소유권', '이전', '상태', '방문', '결과', '차선', '도로', '포장', '바리', '게이트', '포장', '샛길', '이용', '집', '지', '으', '집', '수', '상가', '주택', '상가', '주택', '말', '안', '생각', '잔', '금일', '유예', '민원', '2012년', '6월', '13일', '선', '토지', '사용', '시일', '사용', '수', '최선', '노력', '귀책사유', '지체', '보상금', '지급', '잔', '금일', '수', '답', '토지', '사용', '시기', '2012년', '12월', '잔', '금일', '개월', '전', '책정', '합리', '생각', '잔', '금', '연체', '소유권', '날', '기간', '원', '원', '금액', '토지', '잔', '금일', '6월', '돈', '소유권', '이전', '2014년', '8월', '말', '문제', '공급', '계약서', '토지', '사용', '시기', '년', '하반기', '기재', '공급', '계약서', '계약', '내용', '공고', '문서', '건', 

듯
옆
법
법
일
법
한
길
속
해
생
은
범
시
수
법
글
수
글
['존경', '문', '대통령', '삼성', '총수', '이재용', '부회장', '석방', '아버지', '이건', '회', '회장', '얼마', '아버지', '슬픔', '전', '부회장', '을', '감방', '일', '조선', '시대', '대통령', '이재용', '회장', '석방', '삼성', '대한민국', '경제성장', '이바지', '이', '나라', '경제', '발전', '이', '부회장', '아버지', '이건희', '회장', '이병철', '회장', '서', '이', '나라', '일', '공로', '기억', '하시', '이상', '감옥', '경제', '성장', '석방', '어', '사람', '당시', '박근혜', '대통령', '요청', '을', '하면', '뜻', '수', '존경', '대통령', '경제성장', '이재용', '부회장', '을', '석방', '국가', '기반', '사업', '미래', '성장', '동력', '지금', '이재용', '부회장', '석방', '지금', '이재용', '부회장', '석방', '나라', '나라', '경제', '안위', '이재용', '부회장', '석방', '청원', '존경', '대통령', '아버지', '슬픔', '국가', '경제', '발전', '성장', '도모', '한', '공로', '기억', '하시', '석방', '번', '청원', '대통령', '선처', '재가']
문
회
전
을
일
이
이
서
일
어
을
뜻
을
한
번
['수녀', '전', '강화군', '곳곳', '성토', '작업', '진행', '작년', '강화군', '문제', '제기', '무지', '농민', '불편', '농지', '농지', '성토', '업자', '말', '마음', '농지', '흙', '지금', '단속', '강화군', '금년', '순환', '토사', '수치', '순환', '토사', '지도', '농민', '강화', '군청', '직원', '으름장', '성토', '흙', '문제', '발생', '원

임
속
수
때
수
도
게
식
집
분
뭐
뭐
뭐
게
말
법
바
번
['안녕하세요', '일', '선서', '태권도장', '운영', '관장', '어린이', '보호', '차량', '차량', '검사', '앞', '강화', '소식', '노랑', '차로', '도색', '하차', '확인', '벨', '설치', '경', '등', '설치', '정지', '표지판', '설치', '발판', '설치', '설치', '동승', '자', '정부', '지원', '원', '자비', '말', '아이들', '안전', '지도자', '입장', '바', '어린이집', '지원', '어린이', '대상', '말', '형평', '합기도', '특공', '무술', '등', '동종', '경쟁', '업체', '안전장치', '아이들', '차', '학원', '부모', '제정신', '이', '형평', '위', '언급', '내용', '일전', '이슈', '부분', '정부', '차원', '소통', '일방', '고수', '은', '앞', '강화', '부분', '선팅', '지', '운행', '기록', '장치', '설치', '기존', '차량', '창문', '스티커', '사용', '이제', '안', '선팅', '지도', '검사', '통과', '수', '이야기', '당황', '창문', '일', '절', '스티커', '선팅', '지', '여름', '차량', '열기', '감당', '하', '어', '직사광선', '노출', '아이들', '건강', '생각', '안', '대책', '어이', '한국', '교통안전공단', '홈페이지', '보니', '이유', '아래', '어린이', '운송', '승', '합자', '동차', '도입', '배경', '어린이', '운송', '승', '합자', '동차', '짙은', '차량', '내부', '확인', '상황', '어린이', '방치', '의식', '불명', '등', '어린이', '안전사고', '지속', '발생', '방지', '어린이', '운송', '승', '합자', '동차', '어린이', '하차', '확인', '

...

분
수
분
['대통령', '적', '아시안게임', '올림픽', '엘리트', '체육', '꿈', '꿈', '학교', '현장', '학생', '선수', '지도', '엘리트', '체육', '발전', '최선', '학교', '운동부', '지도자', '제가', '오늘', '글', '이유', '지난', '코로나', '19', '바이러스', '인해', '사회', '삶', '수', '밖', '국민', '어려움', '속', '하루하루', '속', '엘리트', '체육', '상징', '목표', '소년', '체육', '대회', '전국체육대회', '각종', '대회', '취소', '학교', '현장', '학생', '선수', '꿈', '노력', '속', '포기', '오늘', '땀', '훈련', '학생', '선수', '이', '학생', '선수', '자신', '꿈', '미래', '번', '기회', '상징', '의미', '전국소년체육대회', '국체', '전이', '올해', '만큼', '개최', '정부', '문체부', '교육부', '논리', '생각', '전국소년체육대회', '전국체육대회', '체육', '대회', '수', '적', '체육', '자신', '재능', '능력', '평가', '인해', '꿈', '수', '시작', '점', '과정', '성장', '미래', '우리나라', '대표', '선수', '국위', '선양', '체육', '가치', '국민', '단결', '자부심', '수', '인재', '육성', '과정', '생각', '소년', '체육', '대회', '가족', '학교', '지역', '대표', '국가', '대표', '성장', '과정', '대회', '시대', '사회', '흐름', '바', '하', '기본', '가치', '목표', '수', '모드', '사회', '공정', '경쟁', '발전', '건', '이야기', '공정', '사회', '평등', '분배', '기준', '음악', '재능', '과학', '재능', '체육', '재능', '자', '자신', '재능', '발휘', '수', '사회', '기회', 

이
까
한
말
실
한
쯤
게
위
때
위
건
길
태
문
후
안
한
배
피
알
게
뿐
밤
엔
이
만
일
밤
앞
문
말
서
피
거
애
말
말
도
수
거
이
담
채
말
몸
엔
외
명
전
말
월
시
이
시
분
후
후
채
말
팩
일
옷
밤
일
실
열
건
약
게
몸
뿐
서
몸
수
때
말
끝
곳
몸
찜
곳
손
차
이
권
년
일
서
말
몸
번
끝
혈
개
명
칼
중
게
도
못
당
번
곁
거
측
한
곳
이
병
알
월
내
실
글
글
이
말
말
지
한
적
일
단
단
단
명
입
숨
수
하
안
죄
내
맘
채
내
이
안
숨
일
말
다
이
줄
줄
안
월
영
을
은
피
건
말
일
말
말
얼
피
건
말
피
말
월
내
귀
이
것
것
말
것
분
하
등
말
도
이
태
칼
앞
탓
당
후
분
안
지
번
것
뒤
번
시
수
도
말
말
이
시
이
엔
품
맘
이
건
해
이
당
수
분
이
수
수
중
중
일
죄
이
할
일
게
거
요
채
이
한
죄
앞
달
일
일
봄
죄
곁
빌
힘
수
주
요
이
명
상
전
글
이
거
자
만
안
시
내
이
내
이
수
상
볼
이
서
수
일
이
끝
이
허
끝
죄
길
을
수
수
단
글
길
번
맘
이
수
분
이
['영리', '재단', '요양', '병원', '대', '임대', '사업', '얼굴', '재단', '이사장', '한', '평생', '재산', '사업', '시작', '재단', '이사장', '재단', '횡포', '하루아침', '사연', '청원', '글', '울산', '중구', '의료', '재단', '요양', '병원', '소유', '건물', '임대', '이', '건물', '방치', '도심', '흉물', '이', '건물', '소유주', '의료', '재단', '영리', '재단', '소유', '거', '임대', '사업', '수', '일', '뒤', '페이퍼', '컴퍼니', '설립', '대표', '이사', '재단', '부탁', '사건', '발단', '지금', '의료', '재단', '이사장', '당시', '공시지가', '돈', '푼', '주지', '계약서', '계약금', '원', '중도금', '원',

성
법
하
겨
죄
성
애
문
하
이
때
만
하
물
으
하
중
말
은
적
주
정
되
손
은
번
개
일
씨
수
면
임
하
미
윤
하
법
감
적
날
손
이
지
명
때
여
전
시
측
말
라
몽
분
혼
달
분
목
을
도
번
때
핵
때
수
눈
을
때
전
등
말
몸
도
뿐
수
간
의
하
게
수
북
게
짓
말
북
정
북
과
손
안
수
이
자
르
내
자
되
때
전
북
입
일
식
법
사
수
속
컷
간
체
손
체
속
한
말
북
입
조
식
입
말
일
한
일
돈
장
주
일
식
돈
돈
인
돈
게
돈
외
등
돈
뿐
돈
헌
돈
년
수
꿀
적
이
해
해
은
번
수
은
하
수
돈
비
꼴
손
꼴
원
해
원
하
법
적
법
뿐
간
인
집
채
집
때
집
맘
뒤
집
쑈
채
사
말
이
법
수
종
게
삶
이
숲
게
게
이
형
법
바
종
입
이
때
돈
길
간
때
수
게
날
입
하
이
파
파
오
말
글
일
체
하
법
직
직
을
집
임
때
욕
입
안
때
수
수
을
수
혜
수
간
이
죄
발
안
뒤
성
처
게
뜻
왕
이
일
명
적
편
것
하
역
말
씨
을
말
수
드
미
바
등
개
수
말
입
수
수
군
을
게
준
측
수
누
수
간
성
땅
법
때
법
수
듯
때
때
떼
면
떼
떼
법
예
예
법
때
법
이
역
한
대
수
경
민
임
경
민
개
살
땅
똥
말
어
씨
침
눈
들
침
수
침
분
전
농
인
폐
뺨
게
탓
탓
적
짝
때
짜
하
널
칼
깨
춤
귀
수
년
수
분
수
분
이
끝
['부모', '경남', '함양', '곳', '삶', '년', '전', '지하수', '우라늄', '검출', '말', '대대적', '상수도', '공사', '마을', '군민', '전원', '지하수', '상수도', '변경', '동의서', '서약서', '서명', '등', '기록', '문건', '상태', '마을', '주민', '이장', '통합', '신청', '집', '상수도', '설치', '누락', '상수도', '공사', '집', '앞', '라인', '작업', '물', '수', '불편', '심정', '주', '정도', '기간', '동안', '동안', '집'

원
명
돈
풀
꿈
꿈
글
['이', '친부', '이', '방송', '분노', '마음', '길', '고민', '끝', '글', '애', '엄마', '모텔', '영수증', '가방', '이', '생각', '임', '테', '기가', '때', '용서', '사랑', '이', '아빠', '엄마', '때문', '자신', '이', '그날', '신발장', '임신', '테스트', '개', '발견', '그날', '그날', '사실', '이혼', '생각', '제가', '추궁', '집', '밤새', '김', '눈', '이', '옆', '눈', '시간', '다음날', '김', '변명', '부인', '남자', '말', '남자', '이', '도', '남자', '이', '어이', '엄마', '자격', '이', '자격', '마지막', '인사', '해', '이', '인사', '아무도', '이', '안방', '엄마', '엄마', '아빠', '사이', '분위기', '엄마', '엄마', '이', '순간', '지금', '원망', '얘기', '이', '말', '이', '직장', '애', '사람', '돈', '이', '거', '그때', '게', '잠깐', '거', '그때', '밑', '장인', '장모', '그때', '집', '이', '거', '다짐', '정신', '달', '집', '우울증', '대인', '기피증', '아무도', '수', '능력', '쓰레기', '인간', '마음', '안', '생각', '조금', '사람', '일자리', '친구', '남자', '얘기', '대기업', '남자', '나이', '돈', '사람', '이', '아무도', '이', '사람', '아빠', '능력', '아빠', '이', '옷', '사이', '수', '이', '번', '답', '문자', '김', '이듬해', '김', '시간', '이', '본건', '번', '장인', '장모', '남자', '이상', '자리', '알', '방송', '그때', '이', '사람', '아무도', '사실', '혼자', '4월', '쯤', '김', '이', '

이
간
년
한
명
평
건
중
길
['금번', '년', '3월', '31일', '자로', '금융', '위원회', '보도자료', '이번', '년대', '7월', '최고', '금리', '인하', '예정', '서민', '금융', '안정', '최고', '금리', '인하', '적극', '동의', '의견', '다만', '추가', '적용', '시점', '정해', '지지', '금리', '인하', '대부', '중개', '수수료', '현행', '중개', '수수료', '상한', '만원', '이하', '만원', '초과', '인하', '발표', '대부', '중개', '수수료', '인하', '근거', '기술', '내용', '다음', '중개', '모집', '의존도', '업계', '특성', '금전', '대부업', '자', '대부', '중개업', '자게', '수수료', '금액', '지급', '최고', '금리', '대비', '중개', '수수료', '수준', '지속', '유지', '중개업자', '대출', '모집인', '과잉', '모집', '행위', '유발', '과다', '광고', '등', '대부', '업체', '대출', '금리', '인하', '신용', '자', '대출', '위험', '흡수', '능력', '유지', '제약', '요소', '모집', '원가', '상승', '신용', '원가', '측면', '개선', '분별', '대출', '권유', '모집', '행태', '개선', '대부', '업체', '신용', '자', '대출', '여력', '확보', '중개', '수수료', '인하', '추진', '상기', '이유', '대부', '중개', '수수료', '인하', '발표', '상기', '내용', '검토', '추진', '중인', '기관', '관계자', '분', '실질', '문제점', '것', '금리', '인하', '모집', '수수료', '이유', '타당성', '연관', '관계', '전국', '대출', '모집인', '금융사', '금융사', '계약', '수', '탁', '법인', '업체', '소속', '대출', 

In [271]:
len(processing_nouns)

580

In [272]:
processing_nouns

['서울 병원장 일선 의료진 백신 접종 시점 서울 병원장 간호사 화이자 접종 대상 화이자 코로나 직접 격 병동 응급실 경우 해당 화이자 접종 초기 화이자 물량 신청 현 근무자 근무 예정자 배수로 신청 현재 간호사 아래 간호사 의사 명단 토지 공사 범죄 생각 나라 정책 위반 도덕적 해이 병원 장과 간호사 의료진 본인 화이자 송 대상 아니 무능 이고 국민 배신 기만 범죄 지금 당장 교차 접종 요청 화이자 교차 접종 연구 부족 이 게시물 일부 내용 국민 청원 요건 위배 관리자 수정',
 '지금 광양시 광양시 장이 세월 동안 공직 자리 정보 이용 재산 증식 의혹 제기 임직원 주택 사건 하등 사실 실망감 아파트 자리 이해 곳 주민 센터 건립 땅 매입 시민 의견 무시 일 진행 정현 광양시 장과 시장 가족 자제 소유 주변 도시 계획 도로 개설 남은 시장 소유 잔여 부지 일반 상업 지역 사방팔방 도로 가격 상승 불 도로 개설 구간 평 토지 도로 개설 계획 시점 대구 소재 법인 매입 확인 당시 토지 한가운데 도로 금싸라기 변신 대구 사람 구입 과정 정현 시장 공직자 재산 신고 상황 광양시 관내 주택 단지 땅 소유 배경 재산 증식 과정 시민 투명 공개 조사',
 '아이들 안전 안녕하세요 광주 북구 단지 재건축 사업 조합원 세대 재건축 사업 진행 광주 교육청 갑지 행태 고발 광주 북구 단지 초등학교 자리 세대 규모 재건축 사업 공사 진행 공사 기간 발생 분지 학생 편의 학생 인근 초등학교 분산 배치 공사 초등학교 모델링 등교 계획 광주 교육 청서 재건축 사업 완공 학생 일조권 학습 건강 이유 초등학교 광주예술고등학교 예술 부지 이전 예술 부지 경사 차선 도로 초등학생 등하교 위험 장소 자리 광주 교육청 초등학교 이전 증축 비용 단지 재건축 조합 천문학 금액 부담 요구 장휘국 광주 교육감 행정 국장 과장 담당자 조합 임원 조합원 여명 참석 면담 자리 구두 자리 주민 편의 시설 사용 조합원 조건 교육 수장 공인 이신 교육감 기록 효력 생각 단지 재건축 사업 인가 광주 교육청 동의 필

In [283]:
tfidf_vectorizer = TfidfVectorizer(min_df=4)  # 한 번만 나타나는 단어들은 무시
tfidf_matrix = tfidf_vectorizer.fit_transform(processing_nouns)

vector_array = tfidf_matrix.toarray()
nmf = NMF(n_components = 35, random_state = 0)  # 군집하
nmf.fit(vector_array)
features = nmf.transform(vector_array)

normalizer = Normalizer()
norm_features=normalizer.fit_transform(features)

print(norm_features[0:2])

[[3.92350510e-02 2.01134331e-03 0.00000000e+00 7.96736493e-01
  1.01759209e-02 0.00000000e+00 5.58311640e-03 0.00000000e+00
  0.00000000e+00 5.84694528e-01 0.00000000e+00 0.00000000e+00
  3.20785831e-04 0.00000000e+00 6.30945822e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 3.39686492e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 2.53082001e-02 2.10570666e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.09228984e-01 0.00000000e+00 1.68684195e-02
  4.58272885e-02 3.36252352e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 4.68476373e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.00667033e-02 1.74371100e-01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 4.38992223e-02
  0.00000000e+00 2.85229821e-01 2.00653530e-01 0.00000000e+00
  8.01895120e-03 0.00000000e+00 0.00000000e+00 9.04856028e-01
  3.85698396e-02 0.00000000e+00 0.00000000e+00 1.53092390e-01
  0.00000000e+00 0.000

In [284]:
df_features = pd.DataFrame(norm_features,index=data['title'].tolist())

In [285]:
df_features.index[0:5]

Index(['새치기화이자맞은 서울**병원장과 수간호사급에게 az백신 재접종 요청합니다',
       '정현복 광양시장 재산증식 의혹, 전수 조사가 시급합니다.', '제발 우리 아이들의 안전을 지켜주세요', '스미싱/보이스피싱',
       '필리핀 베이비시터(가사도우미) 근무할 수 있게 해주세요.'],
      dtype='object')

In [286]:
def similarity_print(article_title):
    print("# " + article_title)
    
    article_category = data[data['title']==article_title]['category'].values
    
    article = df_features.loc[article_title]

    similarities = df_features.dot(article)
    top = similarities.nlargest(3)

    count = 1

    for i in range(1, len(top)):
        similar_article_category = data[data['title']==top.index[i]]['category'].values

        if similar_article_category == article_category or similar_article_category=='기타':
            print(f"{count}: "+top.index[i])

            count+=1 
        
    print("\n")

    if count==1:    

        print('유사한 청원이 없습니다.\n')

In [288]:
for title in df_features.index[10:20]:
    article_title = title
    similarity_print(article_title)

# 진주항공과학고의 입학에 대한 억울한 부모의 심정
1: 억울하게 뺏긴 제 아이를 찾아주세요
2: 가정폭력 남편에게 아이를 빼았겼습니다.


# 중소벤쳐기업부 연구지원 검증 강화 제언


유사한 청원이 없습니다.

# 부산 해운대 일대 극심한 난개발 빌딩 신축 허가 제한 청원합니다.
1: 수출업체들은 대한민국 국민이 아닌가요? 유예기간을 늘려주시길 간곡히 부탁드립니다.


# 단지 피를 뽑기 위해 존재하는 공혈동물을 보호해주세요.
1: 반려동물을 물건으로 취급하고 생명의 존엄성이 무시되는 동물보호법을 개정해주세요(말티즈 ** 입질대형견 물림사건)


# 미혼모협회 ***의 미혼모와 아이들을 한 기자으로부터 지켜주십시오.
1: 미혼모 블랙리스트를 만들고 후원물품 판매해 착복한 미혼모협회 ‘***’ 대표 ***를 처벌해 주세요.
2: 종합건설업체에서 협력업체에게 행하는 갑질/횡포가 근절되어 선진문화가 자리 잡을 수 있도록 도와주세요.


# 미혼모 블랙리스트를 만들고 후원물품 판매해 착복한 미혼모협회 ‘***’ 대표 ***를 처벌해 주세요.
1: 종합건설업체에서 협력업체에게 행하는 갑질/횡포가 근절되어 선진문화가 자리 잡을 수 있도록 도와주세요.
2: 아파트 관리소장 횡포와 대표회장 갑질을 신고합니다


# 대표적인 남성잡지 *****과 국방부에서 세금을 들여 제작한 *** 폐간, 위문열차와 위문공연 폐지를 요구합니다.
1: 허위논문을 통과시킨 철학연구회와 남성 혐오와 허위사실 유포를 하는 *** 교수님 처벌을 촉구합니다.
2: 한국양성평등교육진흥원의 설립 취지를 망각한 교육을 중단하고 관련자들을 징계하시기 바랍니다.


# 희대의 연쇄성폭행범 신상공개 국민청원 부탁드립니다!


유사한 청원이 없습니다.

# 공공기관 횡포! 언제까지 힘없는 민간 기업은 아무 소리 못내고 당해야 합니까?


유사한 청원이 없습니다.

# LH특검반대 및 김태년 민주당원내대표 사퇴청원
1: 민주당 당원의 투표권을 박탈하려는 민주당 중앙위원회를 징계해주세요.


